**How to use this note book**

1. Run the `Imports and SDK Setup` cell to start
2. Then, run one of the example workflow cells; each is a self contained example

### Imports and SDK Setup

In [ ]:
import logging

logger = logging.getLogger("luminarycloud")
logger.setLevel(logging.ERROR)

import luminarycloud as lc
import luminarycloud._proto.geometry.geometry_pb2 as gpb
import luminarycloud._proto.cad.shape_pb2 as shapepb
import luminarycloud._proto.cad.boolean_pb2 as booleanpb
import luminarycloud._proto.cad.transformation_pb2 as transformationpb
import luminarycloud._proto.base.base_pb2 as basepb
from luminarycloud.params.geometry import shapes, Volume
from luminarycloud.types import Vector3

import uuid
import sys

sys.path.insert(0, "..")
from sdk_util import get_client_for_env
from typing import List


# <internal>
# Please keep the <internal> tags because assistant will use it to filter out the internal code.
api_env_name = "main"

lc.set_default_client(get_client_for_env(env_name=api_env_name))
client = lc.get_default_client()
# </internal>


def print_modify_res(
    volumes: list[Volume], features: list[gpb.Feature], feature_issues: list[gpb.FeatureIssues]
) -> None:
    print("\n\tFeature Issues")
    for fi in feature_issues:
        print(f"\t - {fi} ")
    print("\n\tFeatures")
    for f in features:
        name = f.feature_name if f.feature_name else "Import"
        print(f"\t - {name}\t(Feature ID: {f.id})")
    print("\n\tVolumes")
    for v in volumes:
        print(f"\t - Volume ID: {v.id}")


print(f"SDK is connected to env: {api_env_name}")
print(f"SDK version: {lc.__version__}")
print(f"SDK installation location: {lc.__file__}")

### Create a new project, create a geometry, modify geometry to add a sphere

In [ ]:
project = lc.create_project("Geometry Demo - Pipe CAD (SDK created)")
print(f"created proejct {project.name} ({project.id})")

geometry = project.create_geometry(
    cad_file_path="../../../testdata/cad/pipe.step", name="pipe geo from sdk", scaling=1.0
)
print(f"\ncreated geometry {geometry.name} ({geometry.id})")

# New Approach:  add a sphere to the geometry
sphere = shapes.Sphere(center=Vector3(x=0.0, y=0.0, z=0.0), radius=2.0)
geometry.add_farfield(sphere)

"""
# Legacy approach:
geometry.modify(
    modification=gpb.Modification(
        mod_type=gpb.Modification.MODIFICATION_TYPE_CREATE_FEATURE,
        feature=gpb.Feature(
            id=str(uuid.uuid1()),
            feature_name="Farfield",
            create=gpb.Create(
                sphere=shapepb.Sphere(
                    radius=2.0,
                )
            ),
        ),
    )
)
"""

features = geometry.list_features()
feature_issues = geometry.list_feature_issues()
surfaces, volumes = geometry.list_entities()
print(f"\nmodified geometry, after adding a sphere the geometry has:")
print_modify_res(volumes, features, feature_issues)

### Modify a Geometry that was previously uploaded in the UI

In [ ]:
project = lc.list_projects()[0]
print(f"\nUsing project '{project.name}' ({project.id})")

geometries = project.list_geometries()
print(
    f"\nInitially, there are {len(geometries)} geometries in this project: {[g.id for g in geometries]}"
)

geometry = geometries[0]
print(f"\nUsing geometry '{geometry.name}' ({geometry.id})")


# Add a sphere
geometry.modify(
    modification=gpb.Modification(
        mod_type=gpb.Modification.MODIFICATION_TYPE_CREATE_FEATURE,
        feature=gpb.Feature(
            id=str(uuid.uuid1()),
            feature_name="Outer Sphere",
            create=gpb.Create(
                sphere=shapepb.Sphere(
                    radius=4.0,
                )
            ),
        ),
    )
)
features = geometry.list_features()
feature_issues = geometry.list_feature_issues()
surfaces, volumes = geometry.list_entities()
print(f"\nAfter adding a sphere, the geometry has:")
print_modify_res(volumes, features, feature_issues)

### Multiple ride height geometries

Creates 5 projects, each with a geoemtry originating from the same CAD file, but with varying ride heights.

In [ ]:
ride_heights = [0.0, -1.0, -2.0, -3.0, -5.0]

tunnel_min_point = basepb.Vector3(x=-5.0, y=-5.0, z=0.0)
tunnel_max_point = basepb.Vector3(x=20.0, y=5.0, z=5.0)

cad_path = "../../../testdata/cad/Simplified_Front_Wing_v1_deleted_bodies.step"

# 0 is the volume ID of the wing body
wing_body_volume_id = 0

for rh in ride_heights:
    # Create a project
    project = lc.create_project(f"IntGeo SDK Demo - Ride Height = {rh}")
    print(f"Using project '{project.name}' ({project.id})")

    # Create a geometry by uploading the CAD file
    geometry = project.create_geometry(
        cad_file_path=cad_path,
        name=f"Simplified_Front_Wing_v1_deleted_bodies ride_height={rh}",
        scaling=1.0,
    )
    print(f"\ncreated geometry {geometry.name} ({geometry.id})")

    # Translate the body in the Y direction to get the desired ride height
    geometry.modify(
        modification=gpb.Modification(
            mod_type=gpb.Modification.MODIFICATION_TYPE_CREATE_FEATURE,
            feature=gpb.Feature(
                id=str(uuid.uuid1()),
                feature_name=f"Translate Wing {rh}",
                transform=gpb.Transform(
                    body=[wing_body_volume_id],
                    # translate the body by the ride height adjustment amount in the Y direction
                    translation=transformationpb.Translation(
                        vector=basepb.Vector3(x=0.0, y=rh, z=0.0)
                    ),
                ),
            ),
        )
    )
    features = geometry.list_features()
    feature_issues = geometry.list_feature_issues()
    surfaces, volumes = geometry.list_entities()
    print(f"\nmodified geometry: translated wing body in the Y direction by {rh}")
    print_modify_res(volumes, features, feature_issues)

    # Rotate the wing body about the y-axis
    geometry.modify(
        modification=gpb.Modification(
            mod_type=gpb.Modification.MODIFICATION_TYPE_CREATE_FEATURE,
            feature=gpb.Feature(
                id=str(uuid.uuid1()),
                feature_name=f"Rotate Wing 20 deg about y-axis",
                transform=gpb.Transform(
                    body=[wing_body_volume_id],
                    # rotate the wing body by 20 deg around the y-axis
                    rotation=transformationpb.Rotation(
                        angle=20.0,
                        arbitrary=transformationpb.AnchoredVector3(
                            origin=basepb.Vector3(x=0.0, y=0.0, z=0.0),
                            direction=basepb.Vector3(x=0.0, y=1.0, z=0.0),
                        ),
                    ),
                ),
            ),
        )
    )
    features = geometry.list_features()
    feature_issues = geometry.list_feature_issues()
    surfaces, volumes = geometry.list_entities()
    print(f"\nmodified geometry: rotate Wing about y-axis")
    print_modify_res(volumes, features, feature_issues)

    # Build the tunnel box
    geometry.modify(
        modification=gpb.Modification(
            mod_type=gpb.Modification.MODIFICATION_TYPE_CREATE_FEATURE,
            feature=gpb.Feature(
                id=str(uuid.uuid1()),
                feature_name=f"Build Tunnel Box",
                create=gpb.Create(box=shapepb.Cube(min=tunnel_min_point, max=tunnel_max_point)),
            ),
        )
    )
    surfaces, volumes = geometry.list_entities()
    tunnel_vol_id = int(volumes[-1].id)
    print(f"\nmodified geometry: created tunnel box (with volume ID: {tunnel_vol_id})")
    features = geometry.list_features()
    feature_issues = geometry.list_feature_issues()
    print_modify_res(volumes, features, feature_issues)

    # Subtract the wing volume from the tunnel box volume
    geometry.modify(
        modification=gpb.Modification(
            mod_type=gpb.Modification.MODIFICATION_TYPE_CREATE_FEATURE,
            feature=gpb.Feature(
                id=str(uuid.uuid1()),
                feature_name="Subtract wing volume from the tunnel box volume",
                boolean=gpb.Boolean(
                    reg_subtraction=booleanpb.RegularSubtraction(
                        bodies=[tunnel_vol_id],
                        tools=[wing_body_volume_id],
                        keep_source_bodies=False,
                        keep_tool_bodies=False,
                    )
                ),
            ),
        )
    )

print(f"\nAfter subtracting wing volume from the tunnel box volume, the geometry has:")
features = geometry.list_features()
feature_issues = geometry.list_feature_issues()
surfaces, volumes = geometry.list_entities()
print_modify_res(volumes, features, feature_issues)